In [1]:
import warnings
import pandas as pd
import json
import re
from collections import Counter
# from sklearn.model_selection import train_test_split
import logging
datPath = '/root/autodl-tmp/BertClassNews/data/'

warnings.filterwarnings("ignore")

In [28]:


Trpath = datPath + 'train_cls-sample.txt'
Testpath = datPath + 'dev_cls-sample.txt'

# f = open(pathTr, 'r')
# train_val = f.read()

def read_process_dat(path):
    try:
        with open(path) as f:
            text = f.read()
    except UnicodeDecodeError:        
        with open(path, errors='ignore') as f:
            text = f.read()
    pattern = r'[\t\n]'
    result = re.split(pattern, text)
    contentIndx = 0
    lblIndx = 1
    n = len(result) - 1
    content, label = [], []
    while lblIndx < n:
        content.append(result[contentIndx])
        label.append(result[lblIndx])
        contentIndx += 2
        lblIndx += 2
    df = pd.DataFrame()
    df['tag'] = label
    df['content'] = content
    df['len'] = df['content'].apply(lambda x: sum([i.isalpha() for i in x]))
    df = df[df['len']>27]
    return df

train_val_df = read_process_dat(Trpath)
test_df = read_process_dat(Testpath)

In [33]:
train_val_df.shape, test_df.shape, Counter(train_val_df['tag'].tolist()), Counter(test_df['tag'].tolist())

((32751, 3),
 (4532, 3),
 Counter({'政策类型': 2557,
          '基建民生': 6398,
          '社会热点': 8704,
          '人事任免': 2167,
          '产业金融': 1135,
          '本地旅游': 6880,
          '通报查处': 3097,
          '数据排名': 799,
          '暖新闻': 680,
          '人文历史': 334}),
 Counter({'数据排名': 111,
          '社会热点': 1205,
          '基建民生': 889,
          '本地旅游': 936,
          '政策类型': 370,
          '人事任免': 280,
          '通报查处': 436,
          '暖新闻': 95,
          '产业金融': 171,
          '人文历史': 39}))

In [2]:
# train valid split
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['tag'], random_state=42)
train_df.to_json(datPath + 'train_cls.json')
val_df.to_json(datPath + 'val_cls.json')
test_df.to_json(datPath + 'test_cls.json')

NameError: name 'train_val_df' is not defined

# Read data

In [2]:
import pandas as pd
datPath = '/root/autodl-tmp/BertClassNews/data/'
wtPath = '/root/autodl-tmp/BertClassNews/wt/'

train_df = pd.read_json(datPath + 'train_cls.json')
val_df = pd.read_json(datPath + 'val_cls.json')
test_df = pd.read_json(datPath + 'test_cls.json')

# train_df = pd.read_json(datPath + 'smpl_train.json')
# val_df = pd.read_json(datPath + 'smpl_val.json')
# test_df = pd.read_json(datPath + 'smpl_test.json')

lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}
train_df['tag'] = train_df['tag'].map(lblEncode)
val_df['tag'] = val_df['tag'].map(lblEncode)
test_df['tag'] = test_df['tag'].map(lblEncode)

In [6]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# Load model

In [5]:
# !pip install torch transformers
from transformers import BertTokenizer, BertForSequenceClassification

# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}

model = BertForSequenceClassification.from_pretrained('bert-base-chinese',num_labels=len(lblEncode))
# inputs = tokenizer(text, return_tensors='pt', padding=True)


KeyboardInterrupt: 

In [4]:
reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}
reverse_lblEncode

{0: '本地旅游',
 1: '通报查处',
 2: '基建民生',
 3: '社会热点',
 4: '暖新闻',
 5: '人事任免',
 6: '政策类型',
 7: '产业金融',
 8: '人文历史',
 9: '数据排名'}

In [6]:
lblEncode

{'本地旅游': 0,
 '通报查处': 1,
 '基建民生': 2,
 '社会热点': 3,
 '暖新闻': 4,
 '人事任免': 5,
 '政策类型': 6,
 '产业金融': 7,
 '人文历史': 8,
 '数据排名': 9}

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import sklearn.model_selection
from torch.nn import CrossEntropyLoss

In [8]:
# reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}

class BERTDataset(Dataset):

    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_texts, val_texts, test_texts = train_df['content'].tolist(), val_df['content'].tolist(), test_df['content'].tolist()
train_labels, val_labels, test_labels = train_df['tag'].tolist(), val_df['tag'].tolist(), test_df['tag'].tolist()

# train_texts, val_texts, test_texts = train_df['content'].iloc[:100].tolist(), val_df['content'].iloc[:100].tolist(), test_df['content'].iloc[:100].tolist()
# train_labels, val_labels, test_labels = train_df['tag'].iloc[:100].tolist(), val_df['tag'].iloc[:100].tolist(), test_df['tag'].iloc[:100].tolist()
def generateDataloader(df, lbl, BATCH_SIZE = 16):
    inputs = tokenizer(df, padding=True, truncation=True,return_tensors='pt')
    dataset = BERTDataset(inputs, lbl)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    return dataloader
import time
import numpy as np

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def evaluate(model, dataloader):
    model.eval()
    final_pred = []
    total = 0
    correct = 0
    pred = {i:[] for i in range(len(lblEncode))}
    # pred = {i:[] for i in [0,2,3]}
    with torch.no_grad():

        for batch in dataloader:
            inputs = torch.stack([t for t in batch['input_ids']]).to(device)
            labels = torch.tensor(batch['labels']).to(device)
            attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
            outputs = model(input_ids=inputs, attention_mask=attention_mask,
                      labels=labels)
            predictions = outputs[1].argmax(dim=1)
            for i in range(len(predictions)):
                predRes = predictions[i].item()
                pred[predRes].append((predictions[i] == labels[i]).item())
                final_pred.append((reverse_lblEncode[predRes], reverse_lblEncode[labels[i].item()]))
                # final_pred.append((predRes, labels[i].item()))

            correct += (predictions == labels).sum().item()
            total += labels.shape[0]
      
    accuracy = correct/total
    res = {reverse_lblEncode[i]:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }
  # res = {i:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }

    return accuracy, res, final_pred

def model_train(model, train_dataloader, val_dataloader, iter = 3):
    for epoch in range(3):
      start = time.time()
      losses = 0
      model.train()
      idx = 0
      for batch in train_dataloader:
        inputs, token_type_ids, attention_mask, labels = batch['input_ids'], batch['token_type_ids'], batch['attention_mask'], batch['labels']

        input_ids = torch.stack([t for t in batch['input_ids']]).to(device)

        attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
        labels = torch.tensor(batch['labels']).to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,labels=labels)
        loss = criterion(outputs['logits'],labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses += loss.item()
        acc = (outputs['logits'].argmax(1) == labels).float().mean()
        idx += 1
        curr = time.time()
        timeConsume = curr - start
        if idx % 300 == 0:
            print (f'Epoch {epoch+1}, batch {idx}, so far takes {timeConsume}')
          # los  =outputs['loss']
            print (f'Loss: {losses}')

      print (acc)
      val_acc, val_cat_acc, val_final_pred= evaluate(model, val_dataloader)
      print ('val acc:', val_acc)
        # if idx % 10 == 0:
        #     logging.info(f"Epoch: {epoch}, Batch[{idx}/{len(train_iter)}], "
        #                   f"Train loss :{loss.item():.3f}, Train acc: {acc:.3f}")

      print(f'Epoch {epoch+1} complete, so far takes {timeConsume}')
    return model


def save_model(model, datPath, model_name = 'BertOrigmodelAllDat_v0.pt'):
    # datPath = '/root/wt/'
    # model_save_path  = datPath + 'BertOrigmodelAllDat_v0.pt'
    model_save_path  = datPath + model_name
    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        model.load_state_dict(loaded_paras)
        logging.info("## 成功载入已有模型，进行追加训练......")
    else:
        torch.save(model.state_dict(), model_save_path)
    return model

In [39]:
model = save_model(model, datPath)

# Retrain on the new training set

## Prepare for new dataset

In [9]:
train_dataloader = generateDataloader(train_df['content'].tolist(), train_df['tag'].tolist()) 
val_dataloader = generateDataloader(val_df['content'].tolist(), val_df['tag'].tolist()) 
test_dataloader = generateDataloader(test_df['content'].tolist(), test_df['tag'].tolist()) 

In [63]:
train_accuracy, train_cat_acc, train_pred = evaluate(model, train_dataloader)
print("Train Accuracy:", train_accuracy, "Train Cat Accuracy:", train_cat_acc)

Train Accuracy: 0.08729007633587786 Train Cat Accuracy: {'本地旅游': 0.06707317073170732, '通报查处': 0.0, '基建民生': 0.061068702290076333, '社会热点': 0.6525522041763341, '暖新闻': 0.0, '人事任免': 0.038632045598480054, '政策类型': 0.08110578021475896, '产业金融': 0.040842696629213485, '人文历史': 0.0, '数据排名': nan}


In [73]:
new_model = model_train(new_model, train_dataloader, val_dataloader)

Epoch 1, batch 300, so far takes 102.73140144348145
Loss: 62.12770641036332
Epoch 1, batch 600, so far takes 205.50254940986633
Loss: 121.30345359444618
Epoch 1, batch 900, so far takes 308.28847217559814
Loss: 171.19219447858632
Epoch 1, batch 1200, so far takes 411.175475358963
Loss: 222.6315535181202
Epoch 1, batch 1500, so far takes 514.284298658371
Loss: 268.84038855461404
tensor(1., device='cuda:0')
val acc: 0.8923828423141505
Epoch 1 complete, so far takes 561.3614139556885
Epoch 2, batch 300, so far takes 102.96962356567383
Loss: 62.883980221580714
Epoch 2, batch 600, so far takes 205.7737991809845
Loss: 120.75829038955271
Epoch 2, batch 900, so far takes 308.98069047927856
Loss: 168.42091518919915
Epoch 2, batch 1200, so far takes 411.88878774642944
Loss: 213.88372763851658
Epoch 2, batch 1500, so far takes 514.7534019947052
Loss: 255.612309363205
tensor(1., device='cuda:0')
val acc: 0.8920775454129141
Epoch 2 complete, so far takes 561.9217591285706
Epoch 3, batch 300, so far

In [10]:
new_model = save_model(model, wtPath, model_name = 'BertRetrainNewAllDat_v1.pt')

In [13]:
train_accuracy, train_cat_acc, train_pred = evaluate(new_model, train_dataloader)
print("Train Accuracy:", train_accuracy, "Train Cat Accuracy:", train_cat_acc)

Train Accuracy: 0.9638549618320611 Train Cat Accuracy: {'本地旅游': 0.9931073025335321, '通报查处': 0.9350550702620585, '基建民生': 0.9524882629107981, '社会热点': 0.9898974892289407, '暖新闻': 0.8781302170283807, '人事任免': 0.9832949308755761, '政策类型': 0.9753747323340471, '产业金融': 0.9157549234135668, '人文历史': 0.7053824362606232, '数据排名': 0.8677563150074294}


In [83]:
val_accuracy, val_cat_acc, val_pred = evaluate(new_model, val_dataloader)
print("Val Accuracy:", val_accuracy, "Val Cat Accuracy:", val_cat_acc)

Val Accuracy: 0.8943672721721875 Val Cat Accuracy: {'本地旅游': 0.9584592145015106, '通报查处': 0.8989598811292719, '基建民生': 0.864530225782957, '社会热点': 0.9170644391408115, '暖新闻': 0.6956521739130435, '人事任免': 0.9837209302325581, '政策类型': 0.9168490153172867, '产业金融': 0.7703349282296651, '人文历史': 0.422680412371134, '数据排名': 0.6954022988505747}


In [84]:
test_accuracy, test_cat_acc, test_pred = evaluate(new_model, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)

Test Accuracy: 0.8826125330979699 Test Cat Accuracy: {'本地旅游': 0.9641255605381166, '通报查处': 0.8914893617021277, '基建民生': 0.8492723492723493, '社会热点': 0.9102112676056338, '暖新闻': 0.6333333333333333, '人事任免': 0.9675090252707581, '政策类型': 0.8978328173374613, '产业金融': 0.7604790419161677, '人文历史': 0.423728813559322, '数据排名': 0.6666666666666666}


# Data augmentation

In [11]:
from imblearn.over_sampling import RandomOverSampler

def remove_part_fPred_train(train_df, train_pred):
    cond_acc = []
    for i,j in train_pred:
        cond_acc.append((i==j) |((i!=j)&(j not in ['本地旅游','基建民生','社会热点'])))
    # cond_acc = pd.Series(cond_acc)
    new_train = train_df[cond_acc]
    new_train = new_train.reset_index(drop = True)
    return new_train

def augment_minority_randomSample(train_df, augment_rate):
    oversample = RandomOverSampler(sampling_strategy = augment_rate)
    X_over, y_over = oversample.fit_resample(train_df.drop('tag', axis=1), train_df['tag'])
    X_over['tag'] = y_over
    return X_over

In [10]:
# new_model = save_model(model, wtPath, model_name = 'BertRetrainNewAllDat_v1.pt')

In [14]:
new_train = remove_part_fPred_train(train_df, train_pred)

In [15]:
new_train.to_json(datPath + 'new_train_cls.json')

In [14]:
Counter(new_train['tag'].tolist()), Counter(train_df['tag'].tolist()), reverse_lblEncode

(Counter({0: 5331,
          1: 2477,
          2: 5072,
          3: 6663,
          4: 544,
          5: 1734,
          6: 2046,
          7: 908,
          8: 267,
          9: 639}),
 Counter({0: 5504,
          1: 2477,
          2: 5118,
          3: 6963,
          4: 544,
          5: 1734,
          6: 2046,
          7: 908,
          8: 267,
          9: 639}),
 {0: '本地旅游',
  1: '通报查处',
  2: '基建民生',
  3: '社会热点',
  4: '暖新闻',
  5: '人事任免',
  6: '政策类型',
  7: '产业金融',
  8: '人文历史',
  9: '数据排名'})

In [15]:
new_train_dataloader = generateDataloader(new_train['content'].tolist(), new_train['tag'].tolist()) 
train_accuracy, train_cat_acc, train_pred = evaluate(new_model, new_train_dataloader)
print("Train Accuracy:", train_accuracy, "Train Cat Accuracy:", train_cat_acc)

Train Accuracy: 0.9833339823215607 Train Cat Accuracy: {'本地旅游': 0.9932923420905534, '通报查处': 0.9386199008768585, '基建民生': 0.9956811935610522, '社会热点': 0.9970073320365106, '暖新闻': 1.0, '人事任免': 0.9832949308755761, '政策类型': 0.9795698924731183, '产业金融': 0.9393939393939394, '人文历史': 0.9920318725099602, '数据排名': 0.8984615384615384}


In [64]:
new_model2 = model_train(new_model, new_train_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model2, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model2 = save_model(new_model2, '/root/wt/', model_name = 'BertRetrainNewAllDatRemErrTrain_v2.pt')

## Random upsampling to minority class

In [12]:
augment_rate2 = {0: 5331,1: 2477,2: 5072,3: 6663,4: 544 * 2,5: 1734,6: 2046,7: 908*2,8: 267*3,9: 639*2}
# new_train = remove_part_fPred_train(train_df, train_pred)
new_train = pd.read_json(datPath + 'new_train_cls.json')
new_train = augment_minority_randomSample(new_train, augment_rate2)
new_train_dataloader = generateDataloader(new_train['content'].tolist(), new_train['tag'].tolist()) 

In [65]:
Counter(new_train['tag'])

Counter({0: 5331,
         1: 2477,
         2: 5072,
         3: 6663,
         4: 1632,
         5: 1734,
         6: 2046,
         7: 1816,
         8: 801,
         9: 1917})

In [ ]:
new_model3 = model_train(model, new_train_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model2, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model3 = save_model(new_model3, wtPath, model_name = 'BertRetrainNewAllDatRemErrTrainRandUp_v3.pt')

Epoch 1, batch 300, so far takes 103.42347836494446
Loss: 231.57247819006443
Epoch 1, batch 600, so far takes 207.21534299850464
Loss: 376.84316881000996
Epoch 1, batch 900, so far takes 310.8795609474182
Loss: 483.290787499398
Epoch 1, batch 1200, so far takes 414.6867365837097
Loss: 588.0966288074851
Epoch 1, batch 1500, so far takes 518.4460413455963
Loss: 673.5722362883389
tensor(1., device='cuda:0')
val acc: 0.024576400549534423
Epoch 1 complete, so far takes 611.8735513687134


## Random upsampling to minority class. Retrain for BertRetrainNewAllDat_v1.pt


In [ ]:
model.to(device)
new_model_v2 = model_train(model, new_train_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model_v2, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
# new_model3 = save_model(new_model3, '/root/wt/', model_name = 'BertRetrainNewAllDatRemErrTrainRandUp_v3.pt')

Epoch 1, batch 300, so far takes 99.83909177780151
Loss: 697.2860451936722
Epoch 1, batch 600, so far takes 200.14200615882874
Loss: 1391.9564501047134
Epoch 1, batch 900, so far takes 300.4501922130585
Loss: 2089.886576652527
Epoch 1, batch 1200, so far takes 400.8224024772644
Loss: 2784.9920345544815
Epoch 1, batch 1500, so far takes 501.2716443538666
Loss: 3483.6392674446106
tensor(0.5000, device='cuda:0')
val acc: 0.2544649671805831
Epoch 1 complete, so far takes 591.6769473552704
Epoch 2, batch 300, so far takes 100.44147372245789
Loss: 699.0497311353683
Epoch 2, batch 600, so far takes 200.7929425239563
Loss: 1395.0247613191605


# Train from scratch on augmented dataset

In [ ]:
# new_model = save_model(model, '/root/wt/', model_name = 'BertRetrainNewAllDat_v1.pt')

In [22]:
model.to(device)
new_model4 = model_train(model, new_train_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model4, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model4 = save_model(new_model4, '/root/wt/', model_name = 'BertRetrainNewAllDatRemErrTrainRandUp_v4.pt')

Epoch 1, batch 300, so far takes 99.42090773582458
Loss: 705.8370118141174
Epoch 1, batch 600, so far takes 199.7743377685547
Loss: 1411.5514986515045
Epoch 1, batch 900, so far takes 300.2780954837799
Loss: 2115.0702583789825
Epoch 1, batch 1200, so far takes 400.8581645488739
Loss: 2818.7002758979797
Epoch 1, batch 1500, so far takes 501.4085969924927
Loss: 3521.4392290115356
Epoch 1, batch 1800, so far takes 602.1666820049286
Loss: 4259.40271115303
tensor(0., device='cuda:0')
val acc: 0.11891314303159822
Epoch 1 complete, so far takes 616.7841415405273
Epoch 2, batch 300, so far takes 100.55902981758118
Loss: 707.7108061313629
Epoch 2, batch 600, so far takes 201.13754200935364
Loss: 1414.274089217186
Epoch 2, batch 900, so far takes 301.71672534942627
Loss: 2119.3740879297256
Epoch 2, batch 1200, so far takes 402.30867624282837
Loss: 2821.7742491960526
Epoch 2, batch 1500, so far takes 502.8005893230438
Loss: 3524.414420723915
Epoch 2, batch 1800, so far takes 603.588666677475
Loss

## Random nlpcda to minority class. Retrain for BertRetrainNewAllDat_v1.pt


In [11]:
!pip install nlpcda
# from nlpcda.tools.Simbert import Simbert
# from nlpcda import EDA


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 527 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 123.7 MB/s eta 0:00:01
  Created wheel for nlpcda: filename=nlpcda-2.5.8-py3-none-any.whl size=527645 sha256=62c11ac377c3dfc7ae79d59bf308efaa56fa1da21bf1e6a8b1bf098fb88428c5
  Stored in directory: /root/.cache/pip/wheels/fa/14/ff/34ee134516d01377323c19a6be48d77c082a8c8c623283e80a
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=3cf47c7ff6ab0554a67d3870aca937770186188b088eca5c99b73f56b814b45e
  Stored in directory: /root/.cache/pip/wheels/e1/4b/67/93eb050406bb5774e9537785446258da898a98e92e7e1bbfae
Successfully built nlpcda jieba


In [17]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from nlpcda import Randomword
from nlpcda import Similarword
from nlpcda import Homophone
from nlpcda import RandomDeleteChar
from nlpcda import Ner
from nlpcda import CharPositionExchange
from nlpcda import baidu_translate
from nlpcda import EquivalentChar


def test_Randomword(test_str, create_num=3, change_rate=0.3):
    '''
    随机【（等价）实体】替换，这里是extdata/company.txt ，随机公司实体替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    smw = Randomword(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_Similarword(test_str, create_num=3, change_rate=0.3):
    '''
    随机【同义词】替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    smw = Similarword(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_Homophone(test_str, create_num=3, change_rate=0.1):
    '''
    随机【同意/同音字】替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    hoe = Homophone(create_num=create_num, change_rate=change_rate)
    return hoe.replace(test_str)


def test_RandomDeleteChar(test_str, create_num=3, change_rate=0.1):
    smw = RandomDeleteChar(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)



def test_ner():
    ner = Ner(ner_dir_name='../write',
              ignore_tag_list=['O', 'T'],
              data_augument_tag_list=['Cause', 'Effect'],
              augument_size=3, seed=0)
    data_sentence_arrs, data_label_arrs = ner.augment('../write/1.txt')
    print(data_sentence_arrs, data_label_arrs)


def test_CharPositionExchange(test_str, create_num=10, change_rate=0.5):
    smw = CharPositionExchange(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_baidu_translate():
    a = 'Free translation for each platform'
    s = baidu_translate(a, appid='xxx', secretKey='xxx')
    print(s)


def test_EquivalentChar(test_str, create_num=10, change_rate=0.5):
    s = EquivalentChar(create_num=create_num, change_rate=change_rate)
    return s.replace(test_str)


def test():
    ts = '''这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击'''
    rs1 = test_Randomword(ts)
    rs2 = test_Similarword(ts)
    rs3 = test_Homophone(ts)
    rs4 = test_RandomDeleteChar(ts)
    rs5 = test_EquivalentChar(ts)
    print('随机实体替换>>>>>>')
    for s in rs1:
        print(s)
    print('随机近义词替换>>>>>>')
    for s in rs2:
        print(s)
    print('随机近义字替换>>>>>>')
    for s in rs3:
        print(s)

    print('随机字删除>>>>>>')
    for s in rs4:
        print(s)
    print('等价字替换>>>>>>')
    for s in rs5:
        print(s)


if __name__ == '__main__':
#     # ts = '''今天是2020年3月8日11:40，天气晴朗，天气很不错。'''
#     # rs = EquivalentChar(create_num=3, change_rate=0.5)
#     # res = rs.replace(ts)
#     # print('等价字替换>>>>>>')
#     # for s in res:
#     #     print(s)
    test()

load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/company.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/同义词.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/同音意字.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/等价字.txt done
随机实体替换>>>>>>
这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
这是个实体：中国通信服务；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
这是个实体：瑞丰光电；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
随机近义词替换>>>>>>
这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
这是个实体：58同城；今儿个是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；夫nlpcad包，用来方便一键数据如虎添翼，可行得通加强NLP模型的泛化性质、减少游走不定、抵抗分庭抗礼攻击
这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模的泛化性、减少波动、抵抗对抗攻击
随机近义字替换>>>>>>
这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方

In [12]:
# config = {
#         'model_path': '/root/autodl-tmp/src/chinese_simbert_L-12_H-768_A-12',
#         'CUDA_VISIBLE_DEVICES': '0',
#         'max_len': 32,
#         'seed': 1
# }
# simbert = Simbert(config=config)
# sent = '把我的一个亿存银行安全吗'
# synonyms = simbert.replace(sent=sent, create_num=5)
# print(synonyms)
# '''
# [('我的一个亿，存银行，安全吗', 0.9871675372123718), 
# ('把一个亿存到银行里安全吗', 0.9352194666862488), 
# ('一个亿存银行安全吗', 0.9330801367759705), 
# ('一个亿的存款存银行安全吗', 0.92387855052948),
#  ('我的一千万存到银行安不安全', 0.9014463424682617)]
# '''

In [26]:
from IPython.utils import io

# with io.capture_output() as captured:
#     load('/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/company.txt') 

# print(captured.stdout)

In [ ]:
augment_rate2 = {0: 5331,1: 2477,2: 5072,3: 6663,4: 544 * 2,5: 1734,6: 2046,7: 908*2,8: 267*3,9: 639*2}
def augment_nlpcda(ts):
    smw1 = Randomword(create_num=1, change_rate=0.3)
    smw2 = Similarword(create_num=1, change_rate=0.3)
    hoe = Homophone(create_num=1, change_rate=0.1)
    smw3 = RandomDeleteChar(create_num=1, change_rate=0.1)
    s = EquivalentChar(create_num=2, change_rate=0.5)
    # return s.replace(ts)[0]
    res = s.replace(ts)
    return [smw1.replace(smw2.replace(hoe.replace(smw3.replace(i)[0])[0])[0])[0] for i in res]

def augment_minority_augmentSample(train_df, augment_rate):
    oversample = RandomOverSampler(sampling_strategy = augment_rate)
    X_over, y_over = oversample.fit_resample(train_df.drop('tag', axis=1), train_df['tag'])
    X_over['tag'] = y_over
    return X_over
train_aug = train_df[train_df['tag'].isin([4,7,8,9])]
with io.capture_output() as captured:
    train_aug1 = train_aug.copy()
    train_aug2 = train_aug.copy()
    tmp = train_aug['content'].apply(lambda x: augment_nlpcda(x)).tolist()
    res1 = [i[0] for i in tmp]
    res2 = [i[1] for i in tmp]
    train_aug1['content'] = res1
    train_aug2['content'] = res2
    train_aug_final = pd.concat([train_df, train_aug1, train_aug2],axis=1).sample(frac=1)

In [ ]:
train_aug_final.to_json(datPath, 'new_train_nlpcda.jason')

In [ ]:
new_model_v4 = model_train(new_model, new_train_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model_v4, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model_v4 = save_model(new_model_v4, wtPath, model_name = 'BertRetrainNewAllDatRemErrTrainNlpcdaAug_v4.pt')